### Test Generator
In this part we generate lattice (graph) and introduce X, Y, Z error using depolarization noisy model, which is more close to real world than the independent model favored by surface code.

Here we need to adjust `physical error rate` and `lattice size` to generate different case

Beware that lattice size `L` is just the width/height of grid, and L\*L is not the number of physical qubits. Instead, L\*L\*2 will be physical qubit num.

To generate `Checking matrix` to fit into `PyMatching` lib, we use special way to translate our surface code into check matrix.

In [67]:
# Parameters
# Lattice size, treat it as a square lattice
sizeX = [5]
sizeY = 5

# physical error rate
p_x = 0.05
p_y = 0.05
p_z = 0.05

1. firstly let's generate lattice and add error to the qubit.

In [68]:
import random
import numpy as np
from enum import Enum
from scipy.sparse import csr_matrix

# we need encode an 2D EDGE position to 1D index in the vector
# REMEMBER: x and y start from 0 !!!
# for toric code, always use sizeX*sizeY*2 physical qubits, sizeY*2 row, each row sizeX qubits
def encode_2D_to_1D(x, y, sizeX, sizeY):
    #throw error if x or y is out of range
    if x < 0 or x >= sizeX:
        raise ValueError("x is out of range")
    if y < 0 or y >= sizeY*2:
        raise ValueError("y is out of range")
    return x + y*sizeX

def decode_1D_to_2D(index, sizeX, sizeY):
    #throw error if index is out of range
    if index < 0 or index >= sizeX*sizeY*2:
        raise ValueError("index is out of range")
    return index%sizeX, index//sizeX

def initialize_lattice(sizeX,sizeY):
    return np.zeros(sizeX*sizeY*2, dtype=int)

def introduce_errors(lattice, p_x, p_y, p_z):
    #introduce errors to the lattice
    error_x = np.zeros(len(lattice),dtype=int)
    error_z = np.zeros(len(lattice),dtype=int)
    for i in range(len(lattice)):
        rand = random.random()
        if rand < p_x:
            error_x[i] = 1
            error_z[i] = 0
        elif rand < p_x + p_y:
            error_x[i] = 1
            error_z[i] = 1
        elif rand < p_x + p_y + p_z:
            error_x[i] = 0
            error_z[i] = 1
    return error_z,error_x

2. Use surface/toric code model to detect Z and X error using star and plaquettes stablilisers seperately.

We firstly build a check_matrix, which can serve as a parity check matrix.

In [69]:
# here we set that the left and right are smooth boundary, and the bottom and top are rough boundary.
def build_check_mat_surface(sizeX,sizeY):
    pass
# use toric code, consider periodic boundary condition
def build_check_mat_toric(sizeX,sizeY):
    #first build Z star stabilizer, only odd row have star
    Z_check_matrix = []
    for y in range(1,sizeY*2,2):
        for x in range(0,sizeX):
            Z_stab = np.zeros(sizeX*sizeY*2, dtype=int)
            Z_stab[encode_2D_to_1D(x,y,sizeX,sizeY)] = \
            Z_stab[encode_2D_to_1D((x+1)%sizeX,y,sizeX,sizeY)] = \
            Z_stab[encode_2D_to_1D(x,y-1,sizeX,sizeY)] = \
            Z_stab[encode_2D_to_1D(x,(y+1)%(sizeY*2),sizeX,sizeY)] = 1
            Z_check_matrix.append(Z_stab)
    #then build X plaquetee stabilizer
    X_check_matrix = []
    for y in range(0,sizeY*2,2):
        for x in range(0,sizeX):
            X_stab = np.zeros(sizeX*sizeY*2, dtype=int)
            X_stab[encode_2D_to_1D((x-1)%sizeX,y,sizeX,sizeY)] = \
            X_stab[encode_2D_to_1D(x,y,sizeX,sizeY)] = \
            X_stab[encode_2D_to_1D(x,(y-1)%(sizeY*2),sizeX,sizeY)] = \
            X_stab[encode_2D_to_1D(x,y+1,sizeX,sizeY)] = 1
            X_check_matrix.append(X_stab)
    #use sparse matrix to save memory
    return csr_matrix(Z_check_matrix), csr_matrix(X_check_matrix)

def get_syndrome(error_lattice, check_matrix):
    #get the syndrome of the error lattice
    return check_matrix.dot(error_lattice) % 2


3. Finally write the syndrome and error lattice to file to finish the generation of testpoint.

In [70]:
def write_test_file(error_x,error_z, syndrome_X,syndrome_Z ,sizeX, filename):
    # turn the error lattice into 2D array, write it to a file
    reshape_error_x = error_x.reshape(-1,sizeX)
    file_content = {
        #error_x and z is for correct answer,
        "error_x":reshape_error_x,
        "error_z":error_z.reshape(-1,sizeX),
        #syndrome_x and syndrome_z is for the input of the decoder
        "syndrome_x":syndrome_X.reshape(-1,sizeX),
        "syndrome_z":syndrome_Z.reshape(-1,sizeX),
        "sizeX":sizeX,
        "sizeY":len(reshape_error_x)//2
    }
    np.savez("../code/data/input/"+filename, file_content)
    return file_content

def write_result_file(consume_time,correct_x,correct_z,filename):
    file_content = {
        "time": consume_time,
        "correct_x":correct_x,
        "correct_z":correct_z,
    }
    np.savez("../code/data/output/"+filename, file_content)
    return file_content


Remember that original lattice and syndromes both need writing out, because we need to visualize the logical error rate in "visualizer" part.

In [71]:
# Initialize and introduce errors
lattice = initialize_lattice(sizeX, sizeY)
# get check matrix for stabilisers
z_check_mat, x_check_mat = build_check_mat_toric(sizeX,sizeY)

# introduce error
error_z,error_x = introduce_errors(lattice, p_x, p_y, p_z)

# get syndrome
syndrome_Z = get_syndrome(error_z,z_check_mat)
syndrome_X = get_syndrome(error_x,x_check_mat)

#now make the lattice and syndrome into 2D array and save them
# test_content = write_test_file(error_x,error_z, syndrome_X,syndrome_Z ,sizeX,"test1.npz")

4. Now that we have check matrix, using pymatch to generate result here.

In [72]:
# now since that check_matrix is here, use pymarching lib to get all the statistic
from pymatching import Matching
import time

#set up a timer
start_time = time.time()
# get the matching graph
decoder_result_Z = Matching(z_check_mat).decode(syndrome_Z)
decoder_result_X = Matching(x_check_mat).decode(syndrome_X)
#different syndrome simply mean different code type, like edged surface, toric.
consume_time = time.time() - start_time
print("consume time: ", consume_time)
print("decoder result Z: ", decoder_result_Z)
result_content = write_result_file(consume_time,decoder_result_X.reshape(-1,sizeX)
    ,decoder_result_Z.reshape(-1,sizeX),"pymatch1.npz")

consume time:  0.002000093460083008
decoder result Z:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0]
